In [ ]:
!pip install -q transformers

# New Section

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
# Set up for GPU training

def try_gpu():
  return torch.device("cuda" if torch.cuda.is_available() else "cpu")

device = try_gpu()

In [3]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: Tesla T4


In [4]:
import re
import string
import nltk
# Uncomment to download "stopwords"
nltk.download("stopwords")
from nltk.corpus import stopwords

translator = str.maketrans('', '', string.punctuation)


def clean_text(text: str):

    # lowercase string
    text = text.lower()

    # remove twitter handles (usernames)
    text = re.sub('@[^\s]+', '', text)
    # Isolate and remove punctuations except '?'
    text = re.sub(r'([\'\"\.\(\)\!\?\\\/\,])', r'  ', text)
    text = re.sub(r'[^\w\s\?]', ' ', text)
    # Remove some special characters
    s = re.sub(r'([\;\:\|•«\n])', ' ', text)
    # remove stop words
    #text = " ".join([word for word in str(
     #    text).split() if word not in stop_words])
    text = re.sub(r'([\;\:\|•«\n])', ' ', text)

    # remove urls
    text = re.sub('((www.[^s]+)|(https?://[^s]+))', ' ', text)

    # remove punctuations
    text = text.translate(translator)

    # remove repeating characters
    text = re.sub(r'(.)1+', r'1', text)

    # remove numbers
    text = re.sub('[0-9]+', '', text)
    text = re.sub(r'Γö£┬╗Γö¼Γò£', '', text)
    text = re.sub(r'≡ƒñú ≡ƒñú ≡ƒñú', '', text)

    text = " ".join([word for word in text.split()
                  if word not in stopwords.words('english')
                  or word in ['ya', 'ha', 'go', 'se','ga','a', 'wo', 'not', 'can']])
    # Remove trailing whitespace
    text = re.sub(r'\s+', ' ', text).strip()

    # remove emojis
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    text = re.sub(emoj, '', text)

    return text.strip()


KeyboardInterrupt: 

In [11]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
import pandas as pd

dataset_path_setswana = "/content/drive/MyDrive/SAfriSenti_Dataset/final_isizulu_safrisenti_tweets_clean.csv"
#final_sesotho_safrisenti_tweets_clean_v3.csv"
#dataset_path = "final_all_twi_tweets.csv"
# {negative: 0, positive: 1, neutral: 2}

# Read the CSV file into a pandas DataFrame
setswana_df = pd.read_csv(dataset_path_setswana, sep=";", encoding='cp437', engine='python')
# Define a mapping for the labels
setswana_df.rename(columns={
    'Clean_tweets': 'text',
    'Labels': 'labels'
}, inplace=True)


#setswana_df.rename(columns={
#    'sentence': 'tweets',
#    'Final_labels': 'labels'
#}, inplace=True)

setswana_df.head()

,text,labels
0,mpilo bani ngempela lena ag,positive
1,trying ignore yeses inhliziyo yami ibuhlungu,negative
2,ingalo enkulu kangaka khwapha ngibone,positive
3,kodwa kukhohlakala kuni lokhu ngempela,positive
4,indigenous languages nonsense dare labeled non...,neutral


In [13]:
import pandas as pd

# Load the original CSV file
df = pd.read_csv(dataset_path_setswana)  # Replace with your actual filename

# Randomly sample 3000 tweets without replacement
sampled_df = df.sample(n=3000, random_state=42)

# Save to a new CSV file
sampled_df.to_csv('zulu_tweets.csv', index=False)

print("3000 random tweets saved to 'sampled_tweets.csv'")


ParserError: Error tokenizing data. C error: Expected 1 fields in line 4530, saw 2


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
dataset_path_sepedi = "/content/drive/MyDrive/SAfriSenti_Dataset/final_isizulu_safrisenti_tweets_clean.csv"
sepedi_df = pd.read_csv(dataset_path_sepedi, sep=";", encoding='cp437', engine='python')
sepedi_df.rename(columns={
    'Clean_tweets': 'tweets',
    'Labels': 'labels'
}, inplace=True)


# Filter tweets where the language is Sepedi and exclude those where the predicted language is "multi"
#sepedi_tweets = sepedi_df[(sepedi_df['predict_name'] == 'Sepedi') & (sepedi_df['predict_name'] != 'multi')]

# Extract tweets from the filtered DataFrame
#tweets = sepedi_tweets['tweets'].tolist()

sepedi_df.head()

,tweets,labels
0,mpilo bani ngempela lena ag,positive
1,trying ignore yeses inhliziyo yami ibuhlungu,negative
2,ingalo enkulu kangaka khwapha ngibone,positive
3,kodwa kukhohlakala kuni lokhu ngempela,positive
4,indigenous languages nonsense dare labeled non...,neutral


In [ ]:
# combine the sepedi and setswana language into one dataset
df = pd.concat([setswana_df,], axis=0, ignore_index=True )
print(df.shape )
df

(25203, 2)


,tweets,labels
0,imaculate ayikho enye indlela kube bengizenzel...,negative
1,cabanga nje nisizonda ngempela ≡ƒÿ¡,negative
2,inkinga niphika nabangani benu abanama dildo,negative
3,bantubonke40 when i said the other day itΓÇÖs ...,negative
4,mbsvudu ngokukhulu ukuzithoba cha ≡ƒÖÅ≡ƒÅ┐,negative
...,...,...
25198,am buddahman hashtag sbonelo soze ehlala emont...,positive
25199,ndumnyandu ≡ƒñú ≡ƒñú ≡ƒñú inkinga,neutral
25200,i think i might be against extending grace bet...,neutral
25201,ndingayeka nobobuthongo ndihleli ndiyithanda u...,negative


In [ ]:
df["tweets"] = df["tweets"].apply(lambda x: clean_text(x))
df.head()

,tweets,labels
0,imaculate ayikho enye indlela kube bengizenzel...,negative
1,cabanga nje nisizonda ngempela ƒÿ,negative
2,inkinga niphika nabangani benu abanama dildo,negative
3,bantubonke said day itγçös hot kwathiwa kum im...,negative
4,mbsvudu ngokukhulu ukuzithoba cha ƒöå ƒå,negative


In [ ]:
# Save the modified DataFrame to a new CSV file
#df.to_csv('/content/drive/MyDrive/final/clean_data3.csv', index=False)

In [ ]:
# Drop rows with NaN labels
#df = df.dropna(subset=["labels"])
#df

In [ ]:
# Reading dataframe
#df = pd.read_csv('/traindata1.1.csv', encoding='utf-8')
df['labels'] = df['labels'].apply( lambda x : x.strip() )
df['labels'].unique()
df.head()

,tweets,labels
0,imaculate ayikho enye indlela kube bengizenzel...,negative
1,cabanga nje nisizonda ngempela ≡ƒÿ¡,negative
2,inkinga niphika nabangani benu abanama dildo,negative
3,bantubonke40 when i said the other day itΓÇÖs ...,negative
4,mbsvudu ngokukhulu ukuzithoba cha ≡ƒÖÅ≡ƒÅ┐,negative


In [ ]:
label_mapping = {"neutral": 0, "positive": 1, "negative": 2}

In [ ]:
#df['labels'] = df['labels'].map(label_mapping)

In [ ]:
import pandas as pd
from torch.utils.data import DataLoader
#TensorData, RandomSampler, SequentialSampler

#label_map = {"neutral": 0, "positive": 1, "negative": 2}

# Map the labels to integers using the defined mapping
df["labels"] = df["labels"].map(label_mapping)

# Convert the labels column to integers
df["labels"] = df["labels"].astype(int)

df["tweets"] = df["tweets"].apply(lambda x: clean_text(x))

df.head()


,tweets,labels
0,imaculate ayikho enye indlela kube bengizenzel...,2
1,cabanga nje nisizonda ngempela ƒÿ,2
2,inkinga niphika nabangani benu abanama dildo,2
3,bantubonke said day itγçös hot kwathiwa kum im...,2
4,mbsvudu ngokukhulu ukuzithoba cha ƒöå ƒå,2


In [ ]:
df["tweets"]

,tweets
0,imaculate ayikho enye indlela kube bengizenzel...
1,cabanga nje nisizonda ngempela ƒÿ
2,inkinga niphika nabangani benu abanama dildo
3,bantubonke said day itγçös hot kwathiwa kum im...
4,mbsvudu ngokukhulu ukuzithoba cha ƒöå ƒå
...,...
25198,buddahman hashtag sbonelo soze ehlala emonti ƒ...
25199,ndumnyandu ƒñú ƒñú ƒñú inkinga
25200,think might extending grace bethuna a difficult
25201,ndingayeka nobobuthongo ndihleli ndiyithanda u...


In [ ]:
len(df)

25203

In [ ]:
#!pip install sentencepiece


AfroXLMR Model

In [ ]:
#MAX_LEN = 256
#from transformers import XLMRobertaModel, XLMRobertaTokenizer, AutoTokenizer
#model_name = "Davlan/afro-xlmr-base"
#model = XLMRobertaModel.from_pretrained(model_name)
#tokenizer.model_max_length = 256


AfriBERTa

In [ ]:
#from transformers import AutoTokenizer, AutoModelForTokenClassification
#model_name = "castorini/afriberta_base"
##model = AutoModelForTokenClassification.from_pretrained(model_name)
#tokenizer = AutoTokenizer.from_pretrained(model_name)
#tokenizer.model_max_length = 512


**mBERT Model**


In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

#MAX_LEN = 256
#from transformers import BertTokenizer, BertModel
#model_name = "bert-base-multilingual-cased"
#model = BertModel.from_pretrained(model_name)
#model = AutoModelForMaskedLM.from_pretrained(model_name)
#tokenizer.model_max_length = 512

**AfroLM Model with self-active **learning**

In [ ]:
###MAX_LEN = 256
#from transformers import XLMRobertaModel, XLMRobertaTokenizer
#model_name = "bonadossou/afrolm_active_learning"
#model = XLMRobertaModel.from_pretrained(model_name)
#tokenizer.model_max_length = 256


**XML-R Model**

In [ ]:
#from transformers import AutoTokenizer, AutoModelForMaskedLM
#model_name = "FacebookAI/xlm-roberta-base"
#model = AutoModelForMaskedLM.from_pretrained(model_name)
#microsoft/Multilingual-MiniLM-L12-H384, FacebookAI/xlm-roberta-base
#tokenizer.model_max_length = 512
#Davlan/xlm-roberta-large-finetuned-xhosa
#Davlan/xlm-roberta-base-finetuned-zulu


PuoBERTa

In [ ]:
#from transformers import RobertaTokenizer, RobertaModel

#Load model and tokenizer
#model_name = 'dsfsi/PuoBERTaJW300'
#model = RobertaModel.from_pretrained(model_name)
#tokenizer.model_max_length = 512


TswanaBERTa

ZuBERTa

In [ ]:
##from transformers import pipeline
#from transformers import AutoTokenizer, AutoModelWithLMHead
#model_name = "MoseliMotsoehli/zuBERTa"
#model = AutoModelWithLMHead.from_pretrained(model_name)
#tokenizer = AutoTokenizer.from_pretrained(model_name)

SERENGETI

In [ ]:
#from transformers import AutoTokenizer, AutoModelForMaskedLM
#model_name = "UBC-NLP/serengeti-E110"
#%env OPENAI_API_KEY=XXX
#tokenizer = AutoTokenizer.from_pretrained(model_name)
#model = AutoModelForMaskedLM.from_pretrained(model_name)
#tokenizer.model_max_length = 512


In [ ]:
from transformers import AutoTokenizer

#Load the tokenizer
#tokenizer = AutoTokenizer.from_pretrained(model_name, do_lower_case=True)
#tokenizer = XLMRobertaTokenizer.from_pretrained(model_name, do_lower_case=True)
#tokenizer = BertTokenizer.from_pretrained(model_name, do_lower_case=True)  #mBERT
#tokenizer = AutoTokenizer.from_pretrained(model_name, do_lower_case=True) #XLMR #Serengeti #zuBERTa
tokenizer = RobertaTokenizer.from_pretrained(model_name, do_lower_case=True)


# Create a function to tokenize a set of texts
def preprocessing_for_bert(data):
    """Perform required preprocessing steps for pretrained BERT.
    @param    data (np.array): Array of texts to be processed.
    @return   input_ids (torch.Tensor): Tensor of token ids to be fed to a model.
    @return   attention_masks (torch.Tensor): Tensor of indices specifying which
                  tokens should be attended to by the model.
    """
    # Create empty lists to store outputs
    input_ids = []
    attention_masks = []

    # For every sentence...
    for sent in data:
        # `encode_plus` will:
        #    (1) Tokenize the sentence
        #    (2) Add the `[CLS]` and `[SEP]` token to the start and end
        #    (3) Truncate/Pad sentence to max length
        #    (4) Map tokens to their IDs
        #    (5) Create attention mask
        #    (6) Return a dictionary of outputs
        encoded_sent = tokenizer.encode_plus(
            sent,
            add_special_tokens=True,        # Add `[CLS]` and `[SEP]`
            max_length=MAX_LEN,                  # Max length to truncate/pad
            padding="max_length",        # Pad sentence to max length
            #return_tensors='pt',           # Return PyTorch tensor
            return_attention_mask=True,      # Return attention mask
            truncation=True
            )

        # Add the outputs to the lists
        input_ids.append(encoded_sent.get('input_ids'))
        attention_masks.append(encoded_sent.get('attention_mask'))

    # Convert lists to tensors
    input_ids = torch.tensor(input_ids)
    attention_masks = torch.tensor(attention_masks)

    return input_ids, attention_masks

In [ ]:
# Encode
from transformers import AutoTokenizer
tweets = df["tweets"]
encoded_tweets = [tokenizer.encode(tweet, add_special_tokens=True) for tweet in tweets]

# Find the maximum length
max_len = max([len(sent) for sent in encoded_tweets])
print('Max length: ', max_len)

In [ ]:
# Ensure the 'tweets' column exists in the DataFrame
if "tweets" in df.columns:
    tweets = df["tweets"]

    # Check if there's data in the 'tweets' column
    if len(tweets) > 0:
        encoded_tweets = [tokenizer.encode(tweet, add_special_tokens=True) for tweet in tweets if tweet.strip() != ""]

        # Check if encoded_tweets is not empty
        if encoded_tweets:
            max_len = max([len(sent) for sent in encoded_tweets])
            print('Max length: ', max_len)
        else:
            print("No valid encoded tweets found.")
    else:
        print("No tweets data available.")
else:
    print("Column 'tweets' not found in the DataFrame.")


### Split the dataset

In [ ]:
from sklearn.model_selection import train_test_split

X = df.tweets.values
y = df.labels.values

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20, random_state=2020, shuffle=True)

In [ ]:
# Count the number of positive, negative, and neutral tweets
#"neutral": 0, "positive": 1, "negative": 2
pos_count = sum(y_train == 1)
neu_count = sum(y_train == 0)
neg_count = sum(y_train == 2)

print("Number of POS tweets:", pos_count)
print("Number of NEU tweets:", neu_count)
print("Number of NEG tweets:", neg_count)

In [ ]:
# Specify `MAX_LEN`
#MAX_LEN = 129
MAX_LEN = 256

# Print sentence 0 and its encoded token ids
token_ids, attn_mask = list(preprocessing_for_bert([X[0]]))
#print('Original: ', X[0])
print('Token IDs: ', token_ids.squeeze().numpy())
#print(attn_mask)

# Run function `preprocessing_for_bert` on the train set and the validation set
train_xs, train_masks = preprocessing_for_bert(X_train)
val_xs, val_masks = preprocessing_for_bert(X_val)

In [ ]:
print(train_xs.shape)
#print(torch.tensor(y_train).shape)
#y_train = y_train.astype(np.float64)  # or np.int64, depending on the nature of your data

In [ ]:
list(y_train).count(0), list(y_train).count(1), list(y_train).count(2)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# Convert other data types to torch.Tensor
train_labels = torch.tensor(y_train)
val_labels = torch.tensor(y_val)

# For fine-tuning BERT, the authors recommend a batch size of 16 or 32.
batch_size = 32

# Create the DataLoader for our training set
train_data = TensorDataset(train_xs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# Create the DataLoader for our validation set
val_data = TensorDataset(val_xs, val_masks, val_labels)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)

In [ ]:
print(len(train_data))
print(len(val_data))

In [ ]:
from transformers import AutoModel, AutoConfig
import torch.nn as nn
import torch

class AfroXLMRSenti(nn.Module):
  """AfroXLMR Model for sentiment analysis."""

  def __init__(self, dropout=0.2, freeze=False):
    super(AfroXLMRSenti, self).__init__()
    input_dim, hidden_size, output_dim = 768, 64, 3

    # Get pretrained model
    self.afroxlmr = model

    # Freeze model
    # Load pretrained model configuration
    config = AutoConfig.from_pretrained(model_name, output_hidden_states=True)

    # Load pretrained model
    self.afroxlmr = AutoModel.from_pretrained(model_name, config=config)

    # Instantiate a one layer feed-forward net for classification
    self.sentiment_classifier = nn.Sequential(
        nn.Linear(input_dim, hidden_size),
        nn.ReLU(),
        nn.Linear(hidden_size, output_dim),
        nn.Softmax(dim=1)
        nn.Dropout(dropout)
    )

    # Freeze model
    if freeze == True:
      for param in self.afroxlmr.parameters():
        param.requires_grad = False

  def forward(self, input_ids, attention_mask=None):
    outputs = self.afroxlmr(input_ids, attention_mask)

    # Extract the last hidden state of the token `[CLS]` for classification task
    last_hidden_state_cls = outputs[0][:, 0, :]
    #last_hidden_state_cls = outputs.pooler_output
    #last_hidden_state_cls = self.dropout(last_hidden_state_cls)

    # Feed into classifier to compute logits
    logits = self.sentiment_classifier(last_hidden_state_cls)

    return logits

  @classmethod
  def from_pretrained(cls, checkpoint_path):
    model = cls()
    model.load_state_dict(torch.load(checkpoint_path))
    return model

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

def initialize_model(epochs=2, learning_rate=1e-4):  #1e-4
    """Initialize the Bert Classifier, the optimizer and the learning rate scheduler."""
    # Instantiate Bert Classifier
    senti_classifier = AfroXLMRSenti(model_name, freeze=False)

    # Tell PyTorch to run the model on GPU
    senti_classifier.to(device)

    # Create the optimizer
    optimizer = torch.optim.AdamW(senti_classifier.parameters(),
                      lr=learning_rate,    # Default learning rate 5e-5
                      eps=1e-8    # Default epsilon value
                      )

    # Total number of training steps
    total_steps = len(train_dataloader) * epochs

    # Set up the learning rate scheduler
    scheduler = get_linear_schedule_with_warmup(optimizer,
                                                num_warmup_steps=0, # Default value
                                                num_training_steps=total_steps)
    return senti_classifier, optimizer, scheduler

In [ ]:
import random
import time
import numpy as np

# Specify loss function
loss_fn = nn.CrossEntropyLoss()

def set_seed(seed_value=42):
  """Set seed for reproducibility.
      """
  random.seed(seed_value)
  np.random.seed(seed_value)
  torch.manual_seed(seed_value)
  torch.cuda.manual_seed_all(seed_value)

def train(model, train_dataloader, val_dataloader=None, epochs=4, evaluation=False):
  """Train the BertClassifier model.
      """

  # Start training loop
  print("Start training...\n")
  for epoch_i in range(epochs):
    # =======================================
    #               Training
    # =======================================
    # Print the header of the result table
    print(f"{'Epoch':^7} | {'Batch':^7} | {'Train Loss':^12} | {'Val Loss':^10} | {'Val Acc':^9} | {'Elapsed':^9}")
    print("-"*70)
    # Measure the elapsed time of each epoch
    t0_epoch, t0_batch = time.time(), time.time()
    # Reset tracking variables at the beginning of each epoch
    total_loss, batch_loss, batch_counts = 0, 0, 0
    # Put the model into the training mode
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):
      batch_counts +=1
      # Load batch to GPU
      b_input_ids, b_attn_mask, b_labels = tuple(t.to(device) for t in batch)
      # Zero out any previously calculated gradients
      optimizer.zero_grad()
      # Perform a forward pass. This will return logits.
      logits = model(b_input_ids, b_attn_mask)
      # Compute loss and accumulate the loss values
      loss = loss_fn(logits, b_labels)
      batch_loss += loss.item()
      total_loss += loss.item()
      # Perform a backward pass to calculate gradients
      loss.backward()
      # Clip the norm of the gradients to 1.0 to prevent "exploding gradients"
      torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
      # Update parameters and the learning rate
      optimizer.step()
      scheduler.step()


      # Print the loss values and time elapsed for every 20 batches
      if (step % 20 == 0 and step != 0) or (step == len(train_dataloader) - 1):
        # Calculate time elapsed for 20 batches
        time_elapsed = time.time() - t0_batch
        # Print training results
        print(f"{epoch_i + 1:^7} | {step:^7} | {batch_loss / batch_counts:^12.6f} | {'-':^10} | {'-':^9} | {time_elapsed:^9.2f}")
        # Reset batch tracking variables
        batch_loss, batch_counts = 0, 0
        t0_batch = time.time()
        # Calculate the average loss over the entire training data
        avg_train_loss = total_loss / len(train_dataloader)
        print("-"*70)
    # =======================================
    #               Evaluation
    # =======================================
    if evaluation == True:
      # After the completion of each training epoch, measure the model's performance
      # on our validation set.
      val_loss, val_accuracy = evaluate(model, val_dataloader)
      # Print performance over the entire training data
      time_elapsed = time.time() - t0_epoch
      print(f"{epoch_i + 1:^7} | {'-':^7} | {avg_train_loss:^12.6f} | {val_loss:^10.6f} | {val_accuracy:^9.2f} | {time_elapsed:^9.2f}")
      print("-"*70)
      print("\n")
    print("Training complete!")

    # Save model
    if (epoch_i + 1) % 1 == 0:
      torch.save(model.state_dict(), f"serengeti2-{epoch_i:02}.pth")



def evaluate(model, val_dataloader):
  """After the completion of each training epoch, measure the model's performance
      on our validation set.
      """
  # Put the model into the evaluation mode. The dropout layers are disabled during
  # the test time.
  model.eval()
  # Tracking variables
  val_accuracy = []
  val_loss = []
  # For each batch in our validation set...
  for batch in val_dataloader:
    # Load batch to GPU
    b_input_ids, b_attn_mask, b_labels = tuple(t.to(device) for t in batch)
    # Compute logits
    with torch.no_grad():
      logits = model(b_input_ids, b_attn_mask)
      # Compute loss
      loss = loss_fn(logits, b_labels)
      val_loss.append(loss.item())
      # Get the predictions
      preds = torch.argmax(logits, dim=1).flatten()
      # Calculate the accuracy rate
      accuracy = (preds == b_labels).cpu().numpy().mean() * 100
      val_accuracy.append(accuracy)
      # Compute the average accuracy and loss over the validation set.
  val_loss = np.mean(val_loss)
  val_accuracy = np.mean(val_accuracy)
  return val_loss, val_accuracy

In [ ]:
set_seed(42)    # Set seed for reproducibility
senti_classifier, optimizer, scheduler = initialize_model(epochs=2, learning_rate=0.00001) #
train(senti_classifier, train_dataloader, val_dataloader, epochs=5, evaluation=True)

In [ ]:
import os
import shutil

In [ ]:
import os
print(os.getcwd())

In [ ]:
source_file = "serengeti2-04.pth"
destination_dir = "/content/drive/My Drive/final/serengeti2-04.pth"
shutil.move(source_file, destination_dir)

In [ ]:
model = AfroXLMRSenti.from_pretrained(destination_dir)
model.to(device)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
# Assuming you have already trained the model and obtained the validation dataloader


# Convert validation set predictions and labels to numpy arrays
val_predictions = []
val_labels = []
for batch in val_dataloader:
    b_input_ids, b_attn_mask, b_labels = tuple(t.to(device) for t in batch)
    with torch.no_grad():
        logits = model(b_input_ids, b_attn_mask)
        preds = torch.argmax(logits, dim=1).cpu().numpy()
    val_predictions.extend(preds)
    val_labels.extend(b_labels.cpu().numpy())

# Calculate accuracy
accuracy = accuracy_score(val_labels, val_predictions)

# Calculate F-score (you can specify the average parameter based on your use case)
f_score = f1_score(val_labels, val_predictions, average='weighted')

# Calculate Precision
precission = precision_score(val_labels, val_predictions, average='weighted')

# Calculate recall
recall = recall_score(val_labels, val_predictions, average='weighted')

conf_matrix = confusion_matrix(val_labels, val_predictions)

# Print the results
print(f"Validation Accuracy: {accuracy*100:.2f}")
print(f"Validation F1-score: {f_score*100:.2f}")
print(f"Validation Precission: {precission*100:.2f}")
print(f"Validation Recall: {recall*100:.2f}")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

class_labels = ["Neutral", "Positive", "Negative"]
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", cbar=False,
            xticklabels=class_labels,
            yticklabels=class_labels)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.show()


In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
from scipy.stats import spearmanr

# Assuming you have already trained the model and obtained the validation dataloader

# Convert validation set predictions and labels to numpy arrays
val_predictions = []
val_labels = []
for batch in val_dataloader:
    b_input_ids, b_attn_mask, b_labels = tuple(t.to(device) for t in batch)
    with torch.no_grad():
        logits = model(b_input_ids, b_attn_mask)
        preds = torch.argmax(logits, dim=1).cpu().numpy()
    val_predictions.extend(preds)
    val_labels.extend(b_labels.cpu().numpy())

# Calculate accuracy
accuracy = accuracy_score(val_labels, val_predictions)

# Calculate F-score (you can specify the average parameter based on your use case)
f_score = f1_score(val_labels, val_predictions, average='weighted')

# Calculate Precision
precision = precision_score(val_labels, val_predictions, average='weighted')

# Calculate Recall
recall = recall_score(val_labels, val_predictions, average='weighted')

# Calculate Spearman's ρ
spearman_rho, _ = spearmanr(val_labels, val_predictions)

# Calculate Matthews correlation coefficient (MCC)
def matthews_correlation_coefficient(conf_matrix):
    if conf_matrix.shape[0] == 1 or conf_matrix.shape[1] == 1:
        return 0.0
    else:
        tp, tn, fp, fn = conf_matrix[1, 1], conf_matrix[0, 0], conf_matrix[0, 1], conf_matrix[1, 0]
        mcc = (tp * tn - fp * fn) / ((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))**0.5
        return mcc

conf_matrix = confusion_matrix(val_labels, val_predictions)
mcc = matthews_correlation_coefficient(conf_matrix)

# Print the results
print(f"Validation Accuracy: {accuracy*100:.2f}")
print(f"Validation F1-score: {f_score*100:.2f}")
print(f"Validation Precision: {precision*100:.2f}")
print(f"Validation Recall: {recall*100:.2f}")
print(f"Validation Spearman's ρ: {spearman_rho:.4f}")
print(f"Validation MCC: {mcc:.4f}")


In [ ]:
from sklearn.metrics import classification_report

# Generate classification report
class_report = classification_report(val_labels, val_predictions)

# Print the classification report
print("Classification Report:")
print(class_report)

In [ ]:
import matplotlib.pyplot as plt

# Data
sentiment_classes = ['NSO', 'TSN', 'SOT', 'XHO', 'ZUL']
positive_counts = [8961,3581, 2587, 6657, 19252]  # Adding 0 for ZUL as it is not provided in the data
neutral_counts = [6132,3392, 1892, 6421, 3378]     # Adding 0 for ZUL as it is not provided in the data
negative_counts = [7515,5789, 4521, 12125, 22400]  # Adding 0 for ZUL as it is not provided in the data

# Plotting
fig, ax = plt.subplots(figsize=(10, 6))

bar_width = 0.3
index = range(len(sentiment_classes))

bar1 = ax.bar(index, positive_counts, bar_width, label='Positive', color='g')
bar2 = ax.bar([i + bar_width for i in index], neutral_counts, bar_width, label='Neutral', color='b')
bar3 = ax.bar([i + 2 * bar_width for i in index], negative_counts, bar_width, label='Negative', color='r')

ax.set_xlabel('Sentiment Class')
ax.set_ylabel('Counts')
ax.set_title('Sentiment Class Distribution')
ax.set_xticks([i + bar_width for i in index])
ax.set_xticklabels(sentiment_classes)
ax.legend()

plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Data
classes = ['NSO', 'TSN', 'SOT', 'XHO', 'ZUL']
pos_counts = [3456, 3581, 2587, 6657, 19252]
neu_counts = [5678, 3392, 1892, 6421, 3378]
neg_counts = [7890, 5789, 4521, 12125, 22400]

# Plotting
bar_width = 0.25
index = range(len(classes))

plt.figure(figsize=(10, 6))

plt.bar(index, pos_counts, color='b', width=bar_width, label='Positive')
plt.bar([i + bar_width for i in index], neu_counts, color='g', width=bar_width, label='Neutral')
plt.bar([i + 2*bar_width for i in index], neg_counts, color='r', width=bar_width, label='Negative')

plt.xlabel('Classes')
plt.ylabel('Counts')
plt.title('Sentiment Analysis Results')
plt.xticks([i + bar_width for i in index], classes)
plt.legend()

plt.tight_layout()
plt.show()


Validation Accuracy: 59.11
Validation F1-score: 57.36
Validation Precision: 57.47
Validation Recall: 59.11
Validation Spearman's ρ: 0.4556
Validation MCC: 0.3836

**PUOBERTa**: Setswana: Validation Accuracy: 56.21

---



*   List item
*   List item


Validation F1-score: 54.86
Validation Precision: 54.59
Validation Recall: 56.21
Validation Spearman's ρ: 0.4128
Validation MCC: 0.3204

MBERT: Validation Accuracy: 59.15
Validation F1-score: 56.96
Validation Precision: 57.54
Validation Recall: 59.15
Validation Spearman's ρ: 0.4545
Validation MCC: 0.3773

       LIME AND SHAP

In [ ]:
                                                                                                                    !pip install -q lime shap


In [ ]:
!pip install shap lime

In [ ]:
!pip install transformers

In [ ]:
import numpy as np
from transformers import BertTokenizer, TFBertForSequenceClassification
from lime.lime_text import LimeTextExplainer

# Load pre-trained BERT model
model = TFBertForSequenceClassification.from_pretrained("/content/drive/MyDrive/final/afroxlmr2-09.pth")
tokenizer = BertTokenizer.from_pretrained(model)
# Define the function to predict using BERT model
def predict_proba(texts):
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="tf")
    input_ids = inputs["input_ids"]
    attention_mask = inputs["attention_mask"]
    logits = model(input_ids, attention_mask=attention_mask)[0]
    probabilities = np.exp(logits) / np.exp(logits).sum(axis=1, keepdims=True)
    return probabilities

# Define class names for multiclass
class_names = ["Class 0", "Class 1", "Class 2"]  # Update with your class names

# Create a LIME explainer
explainer = LimeTextExplainer(class_names=class_names, split_expression=' ')

# Sample input texts
texts = ["a huge congratulations to on winning the grammys award for the song bayethe"]
# Explain predictions using LIME
for i in range(len(texts)):
    explanation = explainer.explain_instance(texts[i], predict_proba, num_features=10, num_samples=1000)
    explanation.show_in_notebook()


In [ ]:
import numpy as np
import shap
from transformers import BertTokenizer, TFBertForSequenceClassification

# Load pre-trained BERT model
model = TFBertForSequenceClassification.from_pretrained('bert-base-multilingual-cased')
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

# Define the function to predict using BERT model
def predict_proba(input_ids, attention_mask):
    logits = model(input_ids, attention_mask=attention_mask)[0]
    probabilities = np.exp(logits) / np.exp(logits).sum(axis=1, keepdims=True)
    return probabilities

# Create a SHAP explainer
explainer = shap.Explainer(predict_proba, masker=shap.maskers.Text(tokenizer))

# Sample input texts
texts = ["ke dula ke lapile and i am annoyed", "wena o dula o le busy, you are so boring."]
encoded_inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="tf")
input_ids = encoded_inputs['input_ids']
attention_mask = encoded_inputs['attention_mask']

# Compute SHAP values
shap_values = explainer(input_ids, attention_mask=attention_mask)

# Display SHAP values
shap.summary_plot(shap_values[0], input_ids, feature_names=tokenizer.convert_ids_to_tokens(input_ids[0]))


In [ ]:
import pandas as pd
from transformers import pipeline
from sklearn.metrics import accuracy_score, f1_score
import numpy as np

# Define the number of initial labeled examples and the number of queries in each iteration
initial_labeled_examples = 1000
queries_per_iteration = 50

# Initialize the zero-shot classification pipeline with MBERT
classifier = pipeline("zero-shot-classification", model="UBC-NLP/serengeti-E110")

# Define the sentiment classes
sentiment_classes = ['negative', 'neutral', 'positive']

# Randomly select initial labeled examples
initial_indices = np.random.choice(len(df), initial_labeled_examples, replace=False)

# Initialize labeled and unlabeled datasets
labeled_df = df.iloc[initial_indices].copy()  # Make a copy to avoid SettingWithCopyWarning
unlabeled_df = df.drop(initial_indices)

# Perform active learning iterations
num_iterations = 5  # Adjust as needed
for i in range(num_iterations):
    # Train classifier on labeled data
    labeled_tweets = labeled_df['tweets'].tolist()
    labeled_predictions = classifier(labeled_tweets, candidate_labels=sentiment_classes)
    labeled_df['predicted_sentiment'] = [prediction['labels'][0] for prediction in labeled_predictions]

    # Calculate accuracy on labeled data
    true_labels = labeled_df['predicted_sentiment'].tolist()
    predicted_labels = labeled_df['predicted_sentiment'].tolist()
    accuracy = accuracy_score(true_labels, predicted_labels)
    print(f"Iteration {i+1} - Accuracy on labeled data: {accuracy:.4f}")

    # Select uncertain examples from unlabeled data
    unlabeled_tweets = unlabeled_df['tweets'].tolist()
    uncertainty_scores = []
    for tweet in unlabeled_tweets:
        prediction = classifier(tweet, candidate_labels=sentiment_classes)
        confidence = prediction['scores'][0]  # Assuming 0 index is the highest confidence label
        uncertainty_scores.append(1 - confidence)  # Convert confidence to uncertainty
    uncertainty_indices = np.argsort(uncertainty_scores)[-queries_per_iteration:]

    # Query human for labels
    queried_df = unlabeled_df.iloc[uncertainty_indices]
    queried_tweets = queried_df['tweets'].tolist()
    queried_predictions = classifier(queried_tweets, candidate_labels=sentiment_classes)
    queried_df['predicted_sentiment'] = [prediction['labels'][0] for prediction in queried_predictions]

    # Manually label queried examples and add to labeled dataset
    # For demonstration purposes, this step should be replaced with actual human annotation
    labeled_df = pd.concat([labeled_df, queried_df])

    # Remove queried examples from unlabeled dataset
    unlabeled_df.drop(unlabeled_df.index[uncertainty_indices], inplace=True)

# Final evaluation
# Train classifier on all labeled data
final_labeled_tweets = labeled_df['tweets'].tolist()
final_labeled_predictions = classifier(final_labeled_tweets, candidate_labels=sentiment_classes)
labeled_df['final_predicted_sentiment'] = [prediction['labels'][0] for prediction in final_labeled_predictions]

# Calculate accuracy and F1 score on all labeled data
final_true_labels = labeled_df['true_label'].tolist()
final_predicted_labels = labeled_df['final_predicted_sentiment'].tolist()
accuracy = accuracy_score(final_true_labels, final_predicted_labels)
f1_macro = f1_score(final_true_labels, final_predicted_labels, average='macro')
f1_weighted = f1_score(final_true_labels, final_predicted_labels, average='weighted')

print(f"Final Accuracy: {accuracy:.2f}")
print(f"Final F1 Score (Macro): {f1_macro:.2f}")
print(f"Final F1 Score (Weighted): {f1_weighted:.2f}")

# Optionally, save the final labeled dataset for further analysis or model training
labeled_df.to_csv('final_labeled_dataset.csv', index=False)


Human-centered assessment for Natural Language Processing (NLP) models using eXplainable Artificial Intelligence (XAI) involves evaluating the performance and behavior of NLP models from a human perspective while utilizing XAI techniques to gain insights into the model's decision-making process. This approach aims to make NLP models more transparent, interpretable, and accountable to users.

Human-centered assessment methods for NLP models using XAI can include:

1. **User Studies:** Conducting user studies to assess the model's usability, usefulness, and user satisfaction. Participants can interact with the model and provide feedback on its responses.

2. **Human Evaluation:** Having human annotators evaluate the quality and relevance of model-generated text. Comparing model outputs to human-written text can provide insights into areas where the model performs well or needs improvement.

3. **Explanations:** Employing XAI techniques to generate explanations for model predictions. This helps users understand why the model made a particular decision and builds trust in its outputs.

4. **Model Auditing:** Analyzing the model's biases, fairness, and ethical considerations by assessing its responses across different demographic groups and identifying potential issues.

5. **Scenario Testing:** Presenting the model with specific scenarios and evaluating its responses. This can help uncover potential failures or limitations in the model's understanding.

6. **Adversarial Testing:** Crafting inputs designed to trick the model and assessing its ability to handle unexpected or misleading input.

7. **Feedback Loop:** Creating a continuous feedback loop with users to gather insights about model behavior in real-world usage and iteratively improving the model.

8. **Human-in-the-Loop Systems:** Integrating human reviewers in the decision-making process, especially in critical applications like content moderation, to ensure model outputs align with human values.

By combining human-centered assessment with XAI techniques, researchers and practitioners can not only improve the performance and reliability of NLP models but also enhance their transparency and interpretability, which is crucial for building trust and accountability in AI systems.

XAI using questionnaires is a method to gather user feedback and insights about the interpretability and transparency of AI models. Questionnaires can help researchers and practitioners understand how users perceive and interact with AI systems, their trust levels, and their preferences for explanations. Here's how you can use questionnaires for eXplainable Artificial Intelligence (XAI):

1. **Questionnaire Design:**
   - Design a questionnaire that includes questions related to the model's predictions, explanations, and user perceptions.
   - Include questions about users' trust in the model's predictions, their understanding of the model's decision-making process, and their preferences for different types of explanations.
   - Use Likert scales or multiple-choice questions to capture users' opinions and sentiments.

2. **Explanation Types:**
   - Include questions about the types of explanations users find most helpful. For example, ask whether users prefer textual explanations, visualizations, or feature importance scores.

3. **Model Performance:**
   - Ask users to rate the model's performance in terms of accuracy, reliability, and relevance of predictions.

4. **Trust and Confidence:**
   - Include questions about users' trust in the model's decisions. Ask if users would follow the model's recommendations in real-world scenarios.

5. **Understanding:**
   - Assess users' understanding of the model's decision-making process. Ask if users can explain why the model made a particular prediction.

6. **Preference for Complexity:**
   - Determine users' preference for simple or complex explanations. Ask whether users want detailed technical explanations or high-level summaries.

7. **Bias and Fairness:**
   - Inquire about users' concerns regarding bias and fairness in the model's predictions. Ask if users think the model's decisions are unbiased and fair.

8. **User Satisfaction:**
   - Measure users' overall satisfaction with the explanations provided by the model.

9. **Open-Ended Questions:**
   - Include open-ended questions to allow users to provide qualitative feedback and suggestions for improvement.

10. **Pilot Testing:**
    - Pilot test the questionnaire with a small group of users to identify any issues or ambiguities in the questions.

11. **Distribution:**
    - Distribute the questionnaire to a diverse group of users who have interacted with the AI model. This can be done through email, web forms, or integrated within the application.

12. **Data Analysis:**
    - Analyze the collected questionnaire responses to gain insights into users' perceptions, preferences, and concerns.

13. **Feedback Incorporation:**
    - Use the insights from the questionnaire to improve the model's explanations and user interactions. Address the issues raised by users and iterate on the model accordingly.

14. **Communication:**
    - Share the results of the questionnaire with stakeholders and users. Transparency about user feedback and improvements builds trust.

15. **Regular Assessment:**
    - Periodically repeat the questionnaire to track changes in user perceptions and assess the impact of model refinements.

Using questionnaires for XAI provides a structured way to gather user feedback and insights, enabling researchers and practitioners to make informed decisions about improving model interpretability and transparency based on users' needs and preferences.

Creating a feedback loop with users to gather insights about model behavior in real-world usage and iteratively improving the model involves several steps to ensure effective communication, data collection, and model refinement. Here's how you can achieve this feedback loop:

1. **User Interaction Integration:** Integrate the NLP model into a user-facing application or platform where users can interact with the model. This could be a chatbot, content recommendation system, or any other NLP-powered tool.

2. **Data Collection:** Collect user interactions with the model, including input text, model responses, and user feedback. Store this data securely for analysis and improvement.

3. **User Feedback Collection:** Include mechanisms for users to provide feedback on the model's responses. This could be through rating, comments, or surveys after each interaction.

4. **Feedback Analysis:** Regularly analyze the collected user feedback and interactions. Look for patterns, trends, and common issues raised by users. Identify areas where the model performs well and where it needs improvement.

5. **Model Evaluation:** Use human evaluation and XAI techniques to assess the model's behavior, generate explanations for predictions, and understand its strengths and weaknesses.

6. **Feedback Categorization:** Categorize user feedback into different categories such as false positives, false negatives, misunderstandings, biased responses, and more. This helps in prioritizing improvement areas.

7. **Iteration Planning:** Based on the feedback analysis and model evaluation, plan iterations for model improvement. Identify specific issues to address and set goals for each iteration.

8. **Model Refinement:** Modify the model based on the insights gathered from user feedback. This could involve fine-tuning the model, addressing biases, improving responses, and enhancing overall performance.

9. **Testing:** Before deploying the refined model, conduct thorough testing using a diverse range of scenarios, inputs, and user interactions to ensure that the changes have the desired effect.

10. **Deployment:** Deploy the refined model to the user-facing application or platform, and monitor its behavior in real-world usage.

11. **Feedback Loop Closure:** Continue collecting user feedback and interactions with the refined model. Analyze whether the improvements have led to better user experiences, reduced issues, and increased user satisfaction.

12. **Continuous Improvement:** Iterate through this feedback loop continuously. Regularly gather insights, refine the model, and deploy improved versions to ensure that the model's behavior aligns with user expectations and needs.

13. **Communication:** Keep users informed about the changes you've made based on their feedback. Transparency in model updates builds trust and encourages user engagement.

14. **User Involvement:** Involve users in the improvement process by soliciting their suggestions for enhancements, new features, or addressing specific issues they encounter.

15. **Documentation:** Maintain documentation of user feedback, model improvements, and changes made in each iteration. This documentation helps in tracking progress and sharing insights with stakeholders.

By implementing a well-defined feedback loop, you can continuously gather user insights, enhance model behavior, and provide an NLP-powered solution that aligns with user expectations and needs.